In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import sqrt
from math import floor
from math import ceil
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SimpleExpSmoothing

% matplotlib inline

try:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], 
                        index_col='dt_venda', date_parser=dateparse)
except:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-343.CSV", sep=";", parse_dates=['det_venda'], 
                        index_col='dt_venda', date_parser=dateparse)
    


In [ ]:
dataFrame.head()

In [ ]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

def adicionarIndicesFaltantes(df):
    idx = pd.date_range(min(df.index), dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

def suavizacaoExponencialSimples(dataFrameMercadoria,i):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))
    
    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste))
    rmse_ses = sqrt(mean_squared_error(teste, y_hat_avg['SES'])); 
    print(dataFrameMercadoria)
    
    #dataFrameMercadoria[len(dataFrameMercadoria)]['SES'] = y_hat_avg[0]['SES']
    #arrayDeDataFrame[len(arrayDeDataFrame)-1]['MM'+str(j)] = arrayDeDataFrame[len(arrayDeDataFrame)-1]['qt_vendida'].rolling(j).mean()
    
    #plt.figure(figsize=(16,4))
    #plt.plot(train, label='Train', linewidth=3, markersize=6, marker='o', )
    #plt.plot(teste, label='Test', linewidth=3, markersize=6, marker='o' )
    #plt.plot(y_hat_avg['SES'],linewidth=3, label='SES')
    #plt.legend(loc='best')
    
    return dataFrameMercadoria

dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]

In [ ]:
#arrayMercadoria = dataFrame['cd_mercadoria'].unique()
#arrayEmpresa = [1, 4, 5]
arrayMercadoria = [117661]
arrayEmpresa = [1]

arrayDeDataFrame = []

for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'],ascending=True)
        
        if (len(df) > 0):
            df = adicionarIndicesFaltantes(df) 
            arrayDeDataFrame.append(suavizacaoExponencialSimples(df,i))                
            
        arrayDeDataFrame[len(arrayDeDataFrame)-1].fillna(0, inplace=True)
         